# Bayes Opt Hacking

In [361]:
%load_ext autoreload
%autoreload 2
from belief_active_plot import BeliefActivePlot

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [365]:
# Running functions
args = {
    'net': 'VariationalGaussianProcessRegressor',
    'architecture': ['GraphSAGE', '32', 'activation', 'tanh',
                     'GraphSAGE', '32', 'activation', 'tanh',
                     'GraphSAGE', '32', 'activation', 'tanh'],
    'optimizer': 'Adam', # Adam
    'lr': 1e-4,
    'data': 'mpro_hts', # moonshot_sorted
    'strategy': 'sequential',
    'acquisition': 'ExpectedImprovement',
    'num_thompson_samples': 1000,
    'num_samples': 1000,
    'q': 96,
    'device': 'cuda:0',
    'num_trials': 20, # 9
    'num_rounds': 10, # 80
    'num_epochs': 10, # 200
    'index_provided': True,
    'early_stopping': False,
    'beliefs': ['ThompsonSampling'],
    'annealing': 1.0,
    'n_inducing_points': 100,
    'sample_frac': 0.05, # 1.0
    'update_representation_interval': 20,
}

from types import SimpleNamespace
args = SimpleNamespace(**args)

plot = BeliefActivePlot(
    # net config
    net=args.net,
    architecture=args.architecture,
    n_inducing_points=args.n_inducing_points,
    annealing=args.annealing,

    # optimizer config
    optimizer_type=args.optimizer,
    lr=args.lr,

    # experiment config
    data=args.data,
    sample_frac=args.sample_frac,
    acquisition=args.acquisition,
    num_samples=args.num_samples,
    num_thompson_samples=args.num_thompson_samples,        
    q=args.q,
    early_stopping=args.early_stopping,

    # beliefs
    beliefs=args.beliefs,

    # housekeeping
    device=args.device,
    num_trials=args.num_trials,
    num_rounds=args.num_rounds,
    num_epochs=args.num_epochs,
    update_representation_interval=args.update_representation_interval
)

In [363]:
ds = plot.generate_data()
net = plot.get_net().to('cuda:0')

In [368]:
plot.q = 96
plot.run_trials(ds)

0
0
1
2
3
4
5
6
7
8
9


KeyboardInterrupt: 